In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
users_subscriptions = pd.read_csv('../../files/users_subscriptions.csv', encoding='utf-8', sep=';')

# Pre-process adjustments

In [3]:
# filtering only users with subscription...
user_with_subscription = users_subscriptions.loc[~users_subscriptions['subscription_id'].isna()]

# change datatype of date columns...
date_cols = ['subscription_start_at', 'subscription_end_at', 'signup_at']

for col in date_cols:
    user_with_subscription[col] = pd.to_datetime(user_with_subscription[col])



/tmp/ipykernel_32166/2008841549.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_with_subscription[col] = pd.to_datetime(user_with_subscription[col])
/tmp/ipykernel_32166/2008841549.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_with_subscription[col] = pd.to_datetime(user_with_subscription[col])
/tmp/ipykernel_32166/2008841549.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [4]:
def create_payments_rows(user_id, subscription_id, subscription_type, start_at, end_at, subscription_price):
    # print(start_at, end_at)
    # print(type(start_at), type(end_at))
    end_at = end_at if not(end_at is pd.NaT) else datetime.now().date()
    r = relativedelta(end_at, start_at)
    months_diff = (r.years * 12) + r.months
    # print({
    #     'month_diff': months_diff,
    #     'start': start_at,
    #     'end_at': end_at
    # })

    payment_value = subscription_price if subscription_type == 'Monthly' else subscription_price / 12 

    data = []

    for i in range(1, months_diff + 2):
        payment = {
            'user_id': user_id,
            'subscription_id': subscription_id,
            'subscription_type': subscription_type,
            'subscription_price': subscription_price,
            'payment_date': start_at + relativedelta(months=i-1),
            'payment_number': i,
            'payment_value': payment_value
        }

        data.append(payment)

    return data



def create_payments_rows_by_cashflow(user_id, subscription_id, subscription_type, start_at, end_at, subscription_price):
    end_at = end_at if not(end_at is pd.NaT) else datetime.now().date()
    
    r = relativedelta(end_at, start_at)
    months_diff = (r.years * 12) + r.months
    year_diff = r.years
    time_diff = months_diff if subscription_type == 'Monthly' else year_diff

    payment_value = subscription_price

    data = []

    for i in range(1, time_diff + 2):
        payment = {
            'user_id': user_id,
            'subscription_id': subscription_id,
            'subscription_type': subscription_type,
            'subscription_price': subscription_price,
            'payment_date': start_at + (relativedelta(months=i-1) if subscription_type == 'Monthly' else relativedelta(years=i-1)),
            'payment_number': i,
            'payment_value': payment_value
        }

        data.append(payment)

    return data

# Process starts

In [5]:
accrual_payments_data = []
cashflow_payments_data = []

for i, row in user_with_subscription.iterrows():
    accrual_payments = create_payments_rows(
        user_id = row['user_id'],
        subscription_id = row['subscription_id'], 
        subscription_type = row['subscription_type'],
        start_at = row['subscription_start_at'],
        end_at = row['subscription_end_at'],
        subscription_price = row['subscription_price_eur']
    )

    cashflow_payments = create_payments_rows_by_cashflow(
        user_id = row['user_id'],
        subscription_id = row['subscription_id'], 
        subscription_type = row['subscription_type'],
        start_at = row['subscription_start_at'],
        end_at = row['subscription_end_at'],
        subscription_price = row['subscription_price_eur']
    )

    accrual_payments_data.append(accrual_payments)
    cashflow_payments_data.append(cashflow_payments)

In [6]:
print(f'user_with_subscription size: {len(user_with_subscription)}')
print(f'payments_data size: {len(accrual_payments_data)}')

accrual_basis_all_payments = [dc for l in accrual_payments_data for dc in l]
print(f'payments_df size: {len(accrual_basis_all_payments)}')

accrual_payments_df = pd.DataFrame(accrual_basis_all_payments)

user_with_subscription size: 5350
payments_data size: 5350
payments_df size: 73438


In [7]:
print(f'user_with_subscription size: {len(user_with_subscription)}')
print(f'payments_data size: {len(cashflow_payments_data)}')

cashflow_all_payments = [dc for l in cashflow_payments_data for dc in l]
print(f'payments_df size: {len(cashflow_all_payments)}')

cashflow_payments_df = pd.DataFrame(cashflow_all_payments)

user_with_subscription size: 5350
payments_data size: 5350
payments_df size: 27892


# Create new csv file with payments data

In [8]:
accrual_payments_df.to_csv('../../files/accrual_basis_payments.csv', encoding='utf-8', sep=';', index=False)
cashflow_payments_df.to_csv('../../files/cash_flow_payments.csv', encoding='utf-8', sep=';', index=False)